In [6]:
import pandas
import requests
import time
import re
from bs4 import BeautifulSoup

file_in = pandas.read_csv('./Example File.csv')

# Function to verify if any of the profiles are in edit mode, banned or, private
def check_status(file_in, column):
    '''
        Args:
            file_in: Pandas Dataframe from csv file
            column : Column containing Steam profiles to be checked (May range from 1 to 4 or 1 depending on how data is formatted)

        Outputs:
            private.csv: csv file containing all profiles which are in private mode
            ban.csv    : csv file containing all profiles which have ban records
            edit.csv   : csv file containing all profiles which are in edit mode     
    '''
    start_time = time.time()
    index = column.split(' ')[-1]
    ban = []
    private = []
    edit_data = file_in[file_in[column].str.contains('/edit')]
    for row in range(0, len(file_in)):
        post_profile = requests.get(file_in[column][row])
        # Time-out to avoid spamming
        time.sleep(1)
        profile_data = BeautifulSoup(post_profile.text, 'html.parser')
        if profile_data.find(class_ = 'profile_ban') != None:
            ban_data = {
                'Name': file_in['Full Name ' + str(index)][row],
                'IGN': file_in['In-game Name ' + str(index)][row],
                'ban_record': file_in[column][row]
               }
            ban.append(ban_data)
        elif profile_data.find(class_ = 'profile_private_info') != None:
            private_data = {
                'Name': file_in['Full Name ' + str(index)][row],
                'IGN': file_in['In-game Name ' + str(index)][row],
                'private_account': (file_in[column][row])
            }
            private.append(private_data)
        else:
            pass
    private_pd = pandas.DataFrame(private)
    ban_pd = pandas.DataFrame(ban)
    private_pd.to_csv('./private.csv')    
    ban_pd.to_csv('./ban.csv')
    edit_data.to_csv('./edit.csv')
    print ('---Completed in %s seconds---' % (time.time() - start_time))

# Function to verify if submitted NRIC data is correct
# Function currently not in use due to confidential data
def check_ic(file_in, column):
    '''
        Args:
            file_in: Pandas Dataframe from csv file
            column : Column containing NRIC entries to be checked

        Outputs:
            ic.csv: csv file containing all entries which invalid NRIC entries
    '''
    index = column.split(' ')[-1]
    failed_ic = []
    nric = re.compile(r'[STGstg]\d{7}[A-Za-z]')
    for row in range(0, len(file_in)):
        if not nric.match(str(file_in[column][row])):
            failed_ic_data = {
                'IGN': file_in['In-game Name ' + str(index)][row],
                'Name': file_in['Full Name ' + str(index)][row],                
                'IC': file_in[column][row]                                               
            }
            failed_ic.append(failed_ic_data)
        else:
            pass
    convert_df = pandas.DataFrame(failed_ic)
    convert_df.to_csv('./ic.csv')

check_status(file_in, 'Steam Profile URL 4')
#check_ic(file_in, 'NRIC/FIN 3')

---Completed in 62.166436195373535 seconds---
